Librerias

In [ ]:
import os
from dotenv import load_dotenv
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth


In [ ]:


load_dotenv()

login =  os.getenv('JUMPSELLER_LOGIN')
authtoken =  os.getenv('JUMPSELLER_AUTHTOKEN')

In [ ]:
def obtener_datos(endpoint, login, authtoken):
    datos = []
    page = 1

    while True:
        url = f"https://api.jumpseller.com/v1/{endpoint}.json?page={page}&limit=50"
        r = requests.get(url, auth=HTTPBasicAuth(login, authtoken))
        if r.status_code != 200:
            print(f"❌ Error al obtener {endpoint}:", r.status_code)
            break

        data = r.json()
        if not data:
            break

        datos.extend(data)
        print(f"✅ Página {page} de {endpoint} descargada.")
        page += 1

    return pd.json_normalize(datos)

'0b0829d746a5f80cd474635e4d8d60d2a025bf60ed4c5278e6'